In [1]:
import os.path, csv
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from collections import namedtuple

DistributionRow = namedtuple("DistributionRow", [
    "name", "requests", "p50", "p66", "p75", "p80", "p90", "p95", "p98", "p99", "p100"
])
RequestsRow = namedtuple("RequestsRow", [
    "method", "name", "requests", "failures", "p50", "mean", "min", "max", "avg_content_size", "rps"
])
TestRun = namedtuple("TestRun", [
    "nthreads", "distribution", "requests"
])
TestCollection = namedtuple("TestCollection", [
    "name", "runs"
])

def read_csv(filename):
    with open(filename) as inf:
        inf.readline()
        return [l for l in csv.reader(inf)]

def load_requests(filename):
    result = []
    for row in read_csv(filename):
        method, name, *xs = row
        result.append(RequestsRow(*([method, name] + [float(c) for c in xs])))
    return result

def load_distribution(filename):
    result = []
    for row in read_csv(filename):
        name, *xs = row
        result.append(DistributionRow(*([name] + [float(c) for c in xs])))
    return result

def load_test_collection(name, basedir):
    runs = []
    for run_name in os.listdir(basedir):
        run_dir = os.path.join(basedir, run_name)
        dist_file = os.path.join(run_dir, "locust_distribution.csv")
        req_file = os.path.join(run_dir, "locust_requests.csv")
        if not (os.path.isdir(run_dir) and os.path.isfile(dist_file) and os.path.isfile(req_file)):
            continue
        distribution = load_distribution(dist_file)
        requests = load_requests(req_file)
        runs.append(TestRun(int(run_name), distribution, requests))
    return TestCollection(name, list(sorted(runs)))

TESTS = {}
ALL_THREADCOUNTS = set()
for name in os.listdir("csv"):
    basedir = os.path.join("csv", name)
    if not os.path.isdir(basedir):
        print("not a directory.")
        continue
    tc = load_test_collection(name, basedir)
    if tc and tc.runs:
        TESTS[name] = tc
        ALL_THREADCOUNTS |= {r.nthreads for r in tc.runs}

In [2]:
@interact
def plot_stats_per_threadcount(testcase = TESTS.keys()):
    tc = TESTS[testcase]
    fig, axs = plt.subplots(3, 1, sharex=True)
    plt.rcParams["figure.figsize"] = (10,10)
    axs[0].scatter([r.nthreads for r in tc.runs], [r.distribution[-1].p50 for r in tc.runs], label="Median Response time [ms]")
    axs[0].legend()
    axs[1].scatter([r.nthreads for r in tc.runs], [r.requests[-1].rps for r in tc.runs], label="Req/s")
    axs[1].legend()
    axs[2].scatter([r.nthreads for r in tc.runs], [r.requests[-1].failures*100 // r.requests[-1].requests for r in tc.runs], label="Failure%")
    axs[2].legend()
    axs[2].set_ylim(0, 100)
    plt.show()

interactive(children=(Dropdown(description='testcase', options=('rust-jkrafczyk', 'rust-danielr'), value='rust…

In [3]:
@interact
def plot_comparison(threads = list(sorted(ALL_THREADCOUNTS)), field = ["rps", "p50", "failures"]):
    runs = []
    for tc in TESTS.values():
        run = [r for r in tc.runs if r.nthreads == threads]
        if not run:
            continue
        runs.append((tc.name, run[0]))
    runs = list(sorted(runs))
    
    names = []
    values = []
    for (name, run) in runs:
        if field == "rps":
            v = run.requests[-1].rps
        elif field == "p50":
            v = run.requests[-1].p50
        elif field == "failures":
            v = 100 * run.requests[-1].failures / run.requests[-1].requests
        else:
            raise RuntimeError("Invalid field selected.")
        names.append("{}\n{}".format(name,v))
        values.append(v)
        

    x = np.arange(len(names))
    plt.bar(x, values)
    plt.xticks(x, names)
    plt.show()

interactive(children=(Dropdown(description='threads', options=(50, 100, 200, 300, 500, 750, 1000), value=50), …